In [1]:
from tests.data_generation.stress_tests import run_evseq_stress_test

This is for stress-testing the evSeq code by passing in random inputs with known expected output. We then check to see if the evSeq outputs match the expected.

In [2]:
run_evseq_stress_test(True, True, 
                      keep_output = True,
                      seed = 1)

Working on tests for seed 1...
Errors found for non-max Uncoupled DI02-F02 with seed 1.
Errors found for non-max Uncoupled DI04-F05 with seed 1.
Errors found for non-max Uncoupled DI08-B01 with seed 1.
Errors found for non-max Uncoupled DI01-G04 with seed 1.
Errors found for non-max Uncoupled DI07-H12 with seed 1.
Errors found for non-max Uncoupled DI04-G09 with seed 1.
Errors found for non-max Uncoupled DI02-F11 with seed 1.
Errors found for non-max Uncoupled DI08-A01 with seed 1.
Errors found for non-max Uncoupled DI02-A12 with seed 1.
Errors found for non-max Uncoupled DI07-A11 with seed 1.
Errors found for non-max Uncoupled DI03-H05 with seed 1.
Errors found for non-max Uncoupled DI01-F12 with seed 1.
Errors found for non-max Uncoupled DI08-C11 with seed 1.
Errors found for non-max Uncoupled DI05-H10 with seed 1.
Errors found for non-max Uncoupled DI07-G08 with seed 1.
Errors found for non-max Uncoupled DI08-H09 with seed 1.
Errors found for non-max Uncoupled DI01-F07 with seed 1.


KeyboardInterrupt: 

To do:
1. Build code to test the "max" files coming out
4. Add on code to test for DEAD wells.